# Lab 10: Incident Response Copilot

Build an AI copilot that assists with incident response investigations.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/depalmar/ai_for_the_win/blob/main/notebooks/lab10_ir_copilot.ipynb)

## Learning Objectives
- Conversational IR workflow
- State management for investigations
- Tool-assisted data gathering
- Playbook execution guidance

In [ ]:
from typing import List, Dict, Optional
from dataclasses import dataclass, field
from datetime import datetime
import json

## 1. Investigation State

In [ ]:
@dataclass
class IRState:
    """Incident response investigation state."""
    incident_id: Optional[str] = None
    incident_type: Optional[str] = None
    affected_hosts: List[str] = field(default_factory=list)
    investigated_iocs: List[Dict] = field(default_factory=list)
    actions_taken: List[Dict] = field(default_factory=list)
    timeline: List[Dict] = field(default_factory=list)
    containment_status: str = "none"
    messages: List[Dict] = field(default_factory=list)

class StateManager:
    """Manage investigation state."""
    
    def __init__(self):
        self.state = IRState()
    
    def set_incident(self, incident_id: str, incident_type: str):
        self.state.incident_id = incident_id
        self.state.incident_type = incident_type
        self.add_timeline("Investigation started")
    
    def add_host(self, hostname: str):
        if hostname not in self.state.affected_hosts:
            self.state.affected_hosts.append(hostname)
    
    def add_ioc(self, ioc: str, result: Dict):
        self.state.investigated_iocs.append({
            "ioc": ioc,
            "result": result,
            "timestamp": datetime.now().isoformat()
        })
    
    def add_action(self, action: str, result: str):
        self.state.actions_taken.append({
            "action": action,
            "result": result,
            "timestamp": datetime.now().isoformat()
        })
        self.add_timeline(action)
    
    def add_timeline(self, event: str):
        self.state.timeline.append({
            "event": event,
            "timestamp": datetime.now().isoformat()
        })
    
    def add_message(self, role: str, content: str):
        self.state.messages.append({"role": role, "content": content})

## 2. Copilot Tools

In [ ]:
# Comprehensive mock security data for realistic IR scenarios
# Based on a ransomware incident with lateral movement

MOCK_DATA = {
    "hosts": {
        "WORKSTATION-42": {
            "hostname": "WORKSTATION-42",
            "ip": "192.168.1.42",
            "mac": "00:1A:2B:3C:4D:5E",
            "os": "Windows 10 Enterprise 21H2",
            "user": "jsmith",
            "department": "Engineering",
            "last_seen": "2024-01-15T09:30:00Z",
            "status": "compromised",
            "edr_status": "active",
            "processes": ["outlook.exe", "chrome.exe", "powershell.exe", "rundll32.exe"],
            "recent_alerts": ["ALT-001", "ALT-002", "ALT-003"]
        },
        "WORKSTATION-15": {
            "hostname": "WORKSTATION-15",
            "ip": "192.168.1.15",
            "mac": "00:1A:2B:3C:4D:6F",
            "os": "Windows 10 Enterprise 21H2",
            "user": "mjohnson",
            "department": "Finance",
            "last_seen": "2024-01-15T09:45:00Z",
            "status": "at_risk",
            "edr_status": "active",
            "processes": ["excel.exe", "outlook.exe"],
            "recent_alerts": ["ALT-008"]
        },
        "DC01": {
            "hostname": "DC01",
            "ip": "192.168.1.10",
            "mac": "00:1A:2B:3C:4D:01",
            "os": "Windows Server 2019",
            "role": "Domain Controller",
            "last_seen": "2024-01-15T10:00:00Z",
            "status": "compromised",
            "edr_status": "active",
            "recent_alerts": ["ALT-010", "ALT-011"]
        },
        "FS01": {
            "hostname": "FS01",
            "ip": "192.168.1.20",
            "mac": "00:1A:2B:3C:4D:02",
            "os": "Windows Server 2019",
            "role": "File Server",
            "shares": ["\\\\FS01\\Finance$", "\\\\FS01\\HR$", "\\\\FS01\\Engineering$"],
            "last_seen": "2024-01-15T10:05:00Z",
            "status": "encrypted",
            "edr_status": "offline",
            "recent_alerts": ["ALT-015", "ALT-016"]
        },
        "BACKUP01": {
            "hostname": "BACKUP01",
            "ip": "192.168.1.30",
            "mac": "00:1A:2B:3C:4D:03",
            "os": "Windows Server 2019",
            "role": "Backup Server",
            "last_seen": "2024-01-15T09:55:00Z",
            "status": "compromised",
            "edr_status": "active",
            "recent_alerts": ["ALT-012", "ALT-013"]
        },
        "EXCH01": {
            "hostname": "EXCH01",
            "ip": "192.168.1.25",
            "mac": "00:1A:2B:3C:4D:04",
            "os": "Windows Server 2019",
            "role": "Exchange Server",
            "last_seen": "2024-01-15T10:00:00Z",
            "status": "clean",
            "edr_status": "active",
            "recent_alerts": []
        },
        "WEB01": {
            "hostname": "WEB01",
            "ip": "192.168.1.50",
            "mac": "00:1A:2B:3C:4D:05",
            "os": "Windows Server 2019",
            "role": "Web Server",
            "last_seen": "2024-01-15T10:00:00Z",
            "status": "clean",
            "edr_status": "active",
            "recent_alerts": []
        },
    },
    "users": {
        "jsmith": {
            "username": "jsmith",
            "full_name": "John Smith",
            "email": "jsmith@company.com",
            "department": "Engineering",
            "title": "Senior Developer",
            "manager": "bwilliams",
            "groups": ["Domain Users", "Developers", "VPN Users"],
            "last_logon": "2024-01-15T08:30:00Z",
            "status": "compromised",
            "password_last_set": "2023-11-15",
            "mfa_enabled": True
        },
        "svc_backup": {
            "username": "svc_backup",
            "full_name": "Backup Service Account",
            "type": "service_account",
            "groups": ["Domain Users", "Backup Operators"],
            "last_logon": "2024-01-15T09:30:00Z",
            "status": "compromised",
            "password_last_set": "2022-03-01",
            "mfa_enabled": False,
            "spn": "backup/BACKUP01.company.local"
        },
        "da_admin": {
            "username": "da_admin",
            "full_name": "Domain Admin",
            "type": "admin",
            "groups": ["Domain Admins", "Enterprise Admins"],
            "last_logon": "2024-01-15T09:50:00Z",
            "status": "credentials_stolen",
            "mfa_enabled": True
        },
    },
    "iocs": {
        "185.143.223.47": {
            "type": "ip",
            "classification": "Malicious",
            "confidence": 95,
            "threat_type": "C2 Server",
            "malware_family": "Cobalt Strike",
            "threat_actor": "LockBit Affiliate",
            "first_seen": "2023-06-15",
            "last_seen": "2024-01-15",
            "related_iocs": ["45.33.32.156", "malware-c2.evil.com"],
            "mitre_techniques": ["T1071.001", "T1573.002"]
        },
        "45.33.32.156": {
            "type": "ip",
            "classification": "Malicious",
            "confidence": 92,
            "threat_type": "Exfiltration Server",
            "malware_family": "StealBit",
            "threat_actor": "LockBit Affiliate",
            "first_seen": "2023-11-01",
            "related_iocs": ["185.143.223.47"]
        },
        "d41d8cd98f00b204e9800998ecf8427e": {
            "type": "hash_md5",
            "classification": "Malicious",
            "confidence": 98,
            "file_name": "invoice_q4.docm",
            "malware_family": "Emotet Loader",
            "threat_type": "Initial Access",
            "mitre_techniques": ["T1566.001", "T1059.001"]
        },
        "a1b2c3d4e5f6789012345678abcdef01": {
            "type": "hash_md5",
            "classification": "Malicious",
            "confidence": 99,
            "file_name": "locker.exe",
            "malware_family": "LockBit 3.0",
            "threat_type": "Ransomware",
            "mitre_techniques": ["T1486", "T1490"]
        },
        "malware-c2.evil.com": {
            "type": "domain",
            "classification": "Malicious",
            "confidence": 90,
            "threat_type": "C2 Domain",
            "malware_family": "Cobalt Strike",
            "resolved_ips": ["185.143.223.47"],
            "registrar": "NameCheap",
            "created": "2023-12-01"
        },
    },
    "alerts": {
        "ALT-001": {
            "id": "ALT-001",
            "timestamp": "2024-01-15T08:45:00Z",
            "title": "Suspicious Email Attachment Opened",
            "severity": "HIGH",
            "host": "WORKSTATION-42",
            "user": "jsmith",
            "description": "User opened macro-enabled document invoice_q4.docm from external sender",
            "mitre": ["T1566.001"],
            "status": "open"
        },
        "ALT-002": {
            "id": "ALT-002",
            "timestamp": "2024-01-15T08:46:00Z",
            "title": "Encoded PowerShell Execution",
            "severity": "CRITICAL",
            "host": "WORKSTATION-42",
            "user": "jsmith",
            "description": "WINWORD.EXE spawned powershell.exe with -enc parameter",
            "mitre": ["T1059.001"],
            "cmdline": "powershell.exe -nop -w hidden -enc JABjAGw...",
            "status": "open"
        },
        "ALT-003": {
            "id": "ALT-003",
            "timestamp": "2024-01-15T08:47:00Z",
            "title": "Cobalt Strike Beacon Detected",
            "severity": "CRITICAL",
            "host": "WORKSTATION-42",
            "user": "jsmith",
            "description": "rundll32.exe beaconing to 185.143.223.47:443 every 60 seconds",
            "mitre": ["T1071.001"],
            "c2_ip": "185.143.223.47",
            "status": "open"
        },
        "ALT-008": {
            "id": "ALT-008",
            "timestamp": "2024-01-15T09:15:00Z",
            "title": "Unusual SMB Connection",
            "severity": "MEDIUM",
            "host": "WORKSTATION-15",
            "description": "SMB connection from WORKSTATION-42 to WORKSTATION-15 admin share",
            "mitre": ["T1021.002"],
            "status": "open"
        },
        "ALT-010": {
            "id": "ALT-010",
            "timestamp": "2024-01-15T09:45:00Z",
            "title": "DCSync Attack Detected",
            "severity": "CRITICAL",
            "host": "DC01",
            "user": "svc_backup",
            "description": "Replication request from non-DC host BACKUP01",
            "mitre": ["T1003.006"],
            "status": "open"
        },
        "ALT-011": {
            "id": "ALT-011",
            "timestamp": "2024-01-15T09:50:00Z",
            "title": "Domain Admin Credentials Used",
            "severity": "CRITICAL",
            "host": "DC01",
            "user": "da_admin",
            "description": "Domain admin authenticated from BACKUP01 (unusual source)",
            "mitre": ["T1078.002"],
            "status": "open"
        },
        "ALT-012": {
            "id": "ALT-012",
            "timestamp": "2024-01-15T09:30:00Z",
            "title": "Kerberoasting Detected",
            "severity": "HIGH",
            "host": "BACKUP01",
            "description": "Multiple TGS requests for service accounts with RC4 encryption",
            "mitre": ["T1558.003"],
            "status": "open"
        },
        "ALT-015": {
            "id": "ALT-015",
            "timestamp": "2024-01-15T10:00:00Z",
            "title": "Mass File Encryption",
            "severity": "CRITICAL",
            "host": "FS01",
            "description": "Rapid file modifications detected - .lockbit extension added",
            "mitre": ["T1486"],
            "files_affected": 15234,
            "status": "open"
        },
        "ALT-016": {
            "id": "ALT-016",
            "timestamp": "2024-01-15T10:01:00Z",
            "title": "Shadow Copy Deletion",
            "severity": "CRITICAL",
            "host": "FS01",
            "description": "vssadmin.exe delete shadows /all /quiet executed",
            "mitre": ["T1490"],
            "status": "open"
        },
    },
    "timeline": [
        {"time": "2024-01-15T08:30:00Z", "event": "jsmith logs into WORKSTATION-42", "phase": "normal"},
        {"time": "2024-01-15T08:44:00Z", "event": "Phishing email received from 'accounting@c0mpany.com'", "phase": "initial_access"},
        {"time": "2024-01-15T08:45:00Z", "event": "jsmith opens invoice_q4.docm attachment", "phase": "initial_access"},
        {"time": "2024-01-15T08:46:00Z", "event": "Macro executes encoded PowerShell", "phase": "execution"},
        {"time": "2024-01-15T08:47:00Z", "event": "Cobalt Strike beacon established to 185.143.223.47", "phase": "c2"},
        {"time": "2024-01-15T09:00:00Z", "event": "Discovery commands: whoami, net group, nltest", "phase": "discovery"},
        {"time": "2024-01-15T09:15:00Z", "event": "Lateral movement to WORKSTATION-15 via SMB", "phase": "lateral_movement"},
        {"time": "2024-01-15T09:20:00Z", "event": "PsExec to BACKUP01", "phase": "lateral_movement"},
        {"time": "2024-01-15T09:30:00Z", "event": "Kerberoasting - svc_backup ticket requested", "phase": "credential_access"},
        {"time": "2024-01-15T09:35:00Z", "event": "svc_backup password cracked offline", "phase": "credential_access"},
        {"time": "2024-01-15T09:45:00Z", "event": "DCSync attack from BACKUP01", "phase": "credential_access"},
        {"time": "2024-01-15T09:50:00Z", "event": "Domain Admin credentials obtained", "phase": "privilege_escalation"},
        {"time": "2024-01-15T09:55:00Z", "event": "Data exfiltration to 45.33.32.156 via StealBit", "phase": "exfiltration"},
        {"time": "2024-01-15T10:00:00Z", "event": "LockBit 3.0 deployed to FS01", "phase": "impact"},
        {"time": "2024-01-15T10:01:00Z", "event": "Shadow copies deleted on FS01", "phase": "impact"},
        {"time": "2024-01-15T10:05:00Z", "event": "File encryption completed - ransom note dropped", "phase": "impact"},
    ],
    "playbooks": {
        "ransomware": {
            "name": "Ransomware Response",
            "steps": [
                {"step": 1, "action": "Identify affected systems", "auto": True, "critical": True},
                {"step": 2, "action": "Isolate compromised hosts from network", "auto": False, "critical": True},
                {"step": 3, "action": "Preserve evidence (memory dumps, disk images)", "auto": False, "critical": True},
                {"step": 4, "action": "Identify patient zero and infection vector", "auto": True, "critical": True},
                {"step": 5, "action": "Block C2 IPs at firewall", "auto": True, "critical": True},
                {"step": 6, "action": "Reset compromised credentials", "auto": False, "critical": True},
                {"step": 7, "action": "Check backup integrity", "auto": True, "critical": True},
                {"step": 8, "action": "Scan all endpoints for IOCs", "auto": True, "critical": False},
                {"step": 9, "action": "Notify stakeholders and legal", "auto": False, "critical": True},
                {"step": 10, "action": "Begin restoration from clean backups", "auto": False, "critical": False},
            ]
        }
    }
}

class CopilotTools:
    """Tools available to the IR copilot."""
    
    def __init__(self, state_manager: StateManager):
        self.state = state_manager
        self.isolated_hosts = []
        self.blocked_iocs = []
    
    def get_host_info(self, hostname: str) -> Dict:
        """Get detailed information about a host."""
        hostname_upper = hostname.upper()
        if hostname_upper in MOCK_DATA["hosts"]:
            self.state.add_host(hostname_upper)
            return MOCK_DATA["hosts"][hostname_upper]
        return {"error": f"Host {hostname} not found"}
    
    def get_user_info(self, username: str) -> Dict:
        """Get information about a user account."""
        if username.lower() in MOCK_DATA["users"]:
            return MOCK_DATA["users"][username.lower()]
        return {"error": f"User {username} not found"}
    
    def lookup_ioc(self, ioc: str) -> Dict:
        """Look up IOC in threat intelligence."""
        result = MOCK_DATA["iocs"].get(ioc, {"classification": "Unknown", "message": "IOC not found in threat intel"})
        self.state.add_ioc(ioc, result)
        return result
    
    def get_alert(self, alert_id: str) -> Dict:
        """Get details of a specific alert."""
        return MOCK_DATA["alerts"].get(alert_id, {"error": f"Alert {alert_id} not found"})
    
    def get_alerts_for_host(self, hostname: str) -> List[Dict]:
        """Get all alerts for a specific host."""
        hostname_upper = hostname.upper()
        return [a for a in MOCK_DATA["alerts"].values() if a.get("host") == hostname_upper]
    
    def get_timeline(self, phase: str = None) -> List[Dict]:
        """Get attack timeline, optionally filtered by phase."""
        timeline = MOCK_DATA["timeline"]
        if phase:
            timeline = [e for e in timeline if e.get("phase") == phase]
        return timeline
    
    def get_compromised_hosts(self) -> List[Dict]:
        """Get list of all compromised hosts."""
        return [h for h in MOCK_DATA["hosts"].values() 
                if h.get("status") in ["compromised", "encrypted", "at_risk"]]
    
    def isolate_host(self, hostname: str, confirm: bool = False) -> Dict:
        """Isolate a host from the network."""
        hostname_upper = hostname.upper()
        if hostname_upper not in MOCK_DATA["hosts"]:
            return {"error": f"Host {hostname} not found"}
            
        if not confirm:
            return {
                "requires_confirmation": True,
                "message": f"Confirm isolation of {hostname_upper}? This will disconnect the host from the network.",
                "host_info": MOCK_DATA["hosts"][hostname_upper]
            }
        
        self.isolated_hosts.append(hostname_upper)
        self.state.add_action(f"Isolated host {hostname_upper}", "Success")
        self.state.state.containment_status = "partial" if len(self.isolated_hosts) < 3 else "contained"
        return {"success": True, "message": f"{hostname_upper} isolated from network", "isolated_hosts": self.isolated_hosts}
    
    def block_ioc(self, ioc: str) -> Dict:
        """Block an IOC at the firewall/proxy."""
        if ioc in MOCK_DATA["iocs"]:
            self.blocked_iocs.append(ioc)
            self.state.add_action(f"Blocked IOC {ioc}", "Success")
            return {"success": True, "message": f"IOC {ioc} blocked", "ioc_info": MOCK_DATA["iocs"][ioc]}
        return {"success": True, "message": f"IOC {ioc} blocked (not in threat intel)"}
    
    def get_playbook(self, playbook_name: str) -> Dict:
        """Get IR playbook steps."""
        return MOCK_DATA["playbooks"].get(playbook_name, {"error": f"Playbook {playbook_name} not found"})

# Test tools
manager = StateManager()
tools = CopilotTools(manager)

print("=" * 60)
print("IR COPILOT - INCIDENT DATA LOADED")
print("=" * 60)
print(f"\n📊 Available Data:")
print(f"   Hosts: {len(MOCK_DATA['hosts'])}")
print(f"   Users: {len(MOCK_DATA['users'])}")
print(f"   IOCs: {len(MOCK_DATA['iocs'])}")
print(f"   Alerts: {len(MOCK_DATA['alerts'])}")
print(f"   Timeline Events: {len(MOCK_DATA['timeline'])}")

print("\n🔴 Compromised Hosts:")
for host in tools.get_compromised_hosts():
    print(f"   {host['hostname']}: {host['status']}")

print("\n⚠️  Critical Alerts:")
critical = [a for a in MOCK_DATA['alerts'].values() if a['severity'] == 'CRITICAL']
for alert in critical[:5]:
    print(f"   [{alert['id']}] {alert['title']}")

## 3. IR Copilot

In [ ]:
class IRCopilot:
    """AI-powered incident response copilot."""
    
    def __init__(self):
        self.state_manager = StateManager()
        self.tools = CopilotTools(self.state_manager)
        try:
            from anthropic import Anthropic
            self.client = Anthropic()
            self.available = True
        except:
            self.available = False
    
    def chat(self, message: str) -> str:
        """Process user message and respond."""
        self.state_manager.add_message("user", message)
        
        if not self.available:
            response = self._process_local(message)
        else:
            response = self._process_with_llm(message)
        
        self.state_manager.add_message("assistant", response)
        return response
    
    def _process_local(self, message: str) -> str:
        """Process message without LLM."""
        msg_lower = message.lower()
        
        if "workstation" in msg_lower:
            info = self.tools.get_host_info("WORKSTATION-42")
            return f"Host info retrieved:\n{json.dumps(info, indent=2)}"
        
        if "185.143" in message or "ioc" in msg_lower:
            result = self.tools.lookup_ioc("185.143.223.47")
            return f"IOC lookup result:\n{json.dumps(result, indent=2)}"
        
        if "isolate" in msg_lower:
            return "To isolate a host, please confirm the action. Type 'confirm isolate HOSTNAME'."
        
        if "status" in msg_lower:
            return self.get_status()
        
        return "I can help with: host lookups, IOC analysis, containment actions, and investigation status."
    
    def _process_with_llm(self, message: str) -> str:
        """Process with LLM and tools."""
        # Simplified - in production, use tool_use
        return self._process_local(message)
    
    def get_status(self) -> str:
        """Get investigation status."""
        s = self.state_manager.state
        return f"""
## Investigation Status

- **Incident**: {s.incident_id or 'Not set'}
- **Type**: {s.incident_type or 'Unknown'}
- **Affected Hosts**: {len(s.affected_hosts)}
- **IOCs Investigated**: {len(s.investigated_iocs)}
- **Containment**: {s.containment_status}

### Timeline
""" + "\n".join([f"- {e['event']}" for e in s.timeline[-5:]])

In [ ]:
# Interactive session
copilot = IRCopilot()
copilot.state_manager.set_incident("INC-2024-001", "Malware")

# Simulate conversation
queries = [
    "What happened on WORKSTATION-42?",
    "Look up IOC 185.143.223.47",
    "What's the investigation status?"
]

for q in queries:
    print(f"\nUser: {q}")
    print("-" * 40)
    response = copilot.chat(q)
    print(response)

## 4. Playbook Executor

In [ ]:
PLAYBOOKS = {
    "malware": [
        {"step": 1, "action": "Identify affected hosts", "auto": True},
        {"step": 2, "action": "Isolate from network", "auto": False},
        {"step": 3, "action": "Collect forensic evidence", "auto": False},
        {"step": 4, "action": "Analyze malware sample", "auto": True},
        {"step": 5, "action": "Block IOCs", "auto": True},
        {"step": 6, "action": "Remediate affected systems", "auto": False}
    ]
}

class PlaybookExecutor:
    """Execute IR playbooks."""
    
    def __init__(self, copilot: IRCopilot):
        self.copilot = copilot
        self.current_playbook = None
        self.current_step = 0
    
    def start_playbook(self, playbook_name: str) -> str:
        if playbook_name not in PLAYBOOKS:
            return f"Unknown playbook: {playbook_name}"
        
        self.current_playbook = playbook_name
        self.current_step = 0
        return self.get_next_step()
    
    def get_next_step(self) -> str:
        if not self.current_playbook:
            return "No playbook active"
        
        steps = PLAYBOOKS[self.current_playbook]
        if self.current_step >= len(steps):
            return "Playbook complete!"
        
        step = steps[self.current_step]
        auto = "[AUTO]" if step["auto"] else "[MANUAL]"
        return f"Step {step['step']}: {auto} {step['action']}"
    
    def complete_step(self) -> str:
        self.current_step += 1
        self.copilot.state_manager.add_timeline(
            f"Completed playbook step {self.current_step}"
        )
        return self.get_next_step()

# Test playbook
executor = PlaybookExecutor(copilot)
print(executor.start_playbook("malware"))
print(executor.complete_step())
print(executor.complete_step())

## Summary

We built an IR copilot with:

1. **State Management** - Track investigation progress
2. **Tools** - Host info, IOC lookup, containment
3. **Conversational Interface** - Natural language interaction
4. **Playbook Execution** - Guided response workflows

### Next Steps:
1. Add more data source integrations
2. Implement automated evidence collection
3. Create incident documentation generator